In [ ]:
import time
import pandas as pd
import numpy as np
from scipy import *
from numpy import dot, multiply, diag, power
from numpy import pi, exp, sin, cos, cosh, tanh, real, imag
from numpy.linalg import inv, eig, pinv,norm
from scipy.linalg import svd, svdvals
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from io import BytesIO
from functools import partial
import PIL.Image
from IPython.display import clear_output, Image, display, HTML
from __future__ import print_function
from sklearn import preprocessing  
from sklearn import metrics 
import scipy.io as sio 
import time  
import re

In [ ]:
# parameters
model_name ='modelC' 
T=120; time_step=1/30; 
thres=0.9995 ;gap=0; dimx=9 
patience = 12
n_classes=3
layers=3 
t01=122;t02=t01+T 
num_datacase=6
pred_thres=0.9999 
pred_gap=0
lambda_loss_amount = 0.001 
batch_norm=1   
learning_rate = 0.001
training_iters =5000
batch_size =50
display_step = 300
keep_prob=1
dropout=0 
savepath = '/codes/saved_models/'

In [ ]:
# load datasets
rootpath = '/data/train/time_series_data'
train_data  = np.load(os.path.join(rootpath, 'train_data.npy'   ) )
train_labels =np.load(os.path.join(rootpath, 'train_labels.npy' ) ) 
eval_data=np.load(os.path.join(rootpath, 'eval_data.npy'  ) )
eval_labels=np.load(os.path.join(rootpath, 'eval_labels.npy' ) ) 
testpath ='/data/test/time_series_data/total' 
testX_pred =np.load(os.path.join(testpath, 'testX_pred.npy' ) )
testY_pred=np.load(os.path.join(testpath, 'testY_pred.npy'  ) ) 
testX_no=np.load(os.path.join(testpath, 'testX_no.npy' ) )
testY_no=np.load(os.path.join(testpath, 'testY_no.npy' ) ) 

In [ ]:
def conv2d(x, W, b, stride_row,stride_col):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, stride_row,stride_col, 1], padding='VALID') 
    x = tf.nn.bias_add(x, b)  
    return tf.nn.relu(x)  

def batch_norm(x, n_out, phase_train):
    """
    Batch normalization on convolutional maps.
    Ref.: http://stackoverflow.com/questions/33949786/how-could-i-use-batch-normalization-in-tensorflow
    Args:
        x:           Tensor, 4D BHWD input maps
        n_out:       integer, depth of input maps
        phase_train: boolean tf.Varialbe, true indicates training phase
        scope:       string, variable scope
    Return:
        normed:      batch-normalized maps
    """
    with tf.variable_scope('bn'):
        beta = tf.Variable(tf.constant(0.0, shape=[n_out]),
                                     name='beta', trainable=True)
        gamma = tf.Variable(tf.constant(1.0, shape=[n_out]),
                                      name='gamma', trainable=True)
        batch_mean, batch_var = tf.nn.moments(x, [0], name='moments')
        ema = tf.train.ExponentialMovingAverage(decay=0.5)

        def mean_var_with_update():
            ema_apply_op = ema.apply([batch_mean, batch_var])
            with tf.control_dependencies([ema_apply_op]):
                return tf.identity(batch_mean), tf.identity(batch_var)

        mean, var = tf.cond(phase_train,
                            mean_var_with_update,
                            lambda: (ema.average(batch_mean), ema.average(batch_var)))
        normed = tf.nn.batch_normalization(x, mean, var, beta, gamma, 1e-3)
    return normed

def conv2d_norm(x,W,b, phase_train,stride_row,stride_col):
    x = tf.nn.conv2d(x, W, strides=[1, stride_row,stride_col, 1], padding='VALID')  
    x = tf.nn.bias_add(x, b)  
    x_out = batch_norm(x,1,phase_train)
    return tf.nn.relu(x_out ) 

def maxpool2d(x, height,width):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, height,  width, 1], strides=[1,  height,width, 1],
                          padding='SAME') # 
def input_weight_all(Theta,name):# Theta is a list type
    import pickle
    filepointer=open(name,"wb")
    pickle.dump(Theta,filepointer,protocol=2)
    filepointer.close()
    return 

def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
            tf.summary.scalar('stddev', stddev)
            tf.summary.scalar('max', tf.reduce_max(var))
            tf.summary.scalar('min', tf.reduce_min(var))
            tf.summary.histogram('histogram', var) 
# Create model
def conv_net( x,   batch_norm,phase_train):  
    global layers, dimx, keep_prob  
    dep1=8;dep2=16;dep3=16; 
    x = tf.reshape(x, shape=[-1,214,30,1])  
    with tf.variable_scope('eigen_conv1'): 
        wc1= tf.get_variable( 'weight1',shape = [5,5,1,dep1],
        initializer = tf.contrib.layers.xavier_initializer_conv2d(uniform=True, seed=None, dtype=tf.float32)) 
        bc1=tf.get_variable( 'bias1',
          shape = [dep1],
          initializer=tf.constant_initializer(0.0)) 
        stride_row=2;stride_col=2
        if batch_norm:
            conv1 = conv2d_norm(x,wc1,bc1,phase_train,stride_row,stride_col) 
            conv1 = maxpool2d(conv1,2,2)    
        else:
            conv1 = conv2d(x, wc1, bc1,stride_row,stride_col)  
            conv1 = maxpool2d(conv1,2,2 )  
        variable_summaries(wc1)
        variable_summaries(bc1)  
    with tf.variable_scope('eigen_conv2'): 
        wc2= tf.get_variable( 'weight2',shape = [3,3, dep1, dep2],
        initializer = tf.contrib.layers.xavier_initializer_conv2d(uniform=True, seed=None, dtype=tf.float32)) 
        bc2=tf.get_variable( 'bias2',
          shape = [dep2],
          initializer=tf.constant_initializer(0.0)) 
        stride_row=2;stride_col=2
        if batch_norm:
            conv2 = conv2d_norm(conv1,wc2,bc2, phase_train,stride_row,stride_col)
            conv2 = maxpool2d(conv2, 2,2)   
        else:
            conv2 = conv2d(conv1, wc2, bc2,stride_row,stride_col) 
            conv2 = maxpool2d(conv2, 2,2) 
        variable_summaries(wc2)
        variable_summaries(bc2)      
    with tf.variable_scope('eigen_conv3'): 
        wc3= tf.get_variable( 'weight3',shape = [2,2, dep2, dep3],
        initializer = tf.contrib.layers.xavier_initializer_conv2d(uniform=True, seed=None, dtype=tf.float32))  
        bc3=tf.get_variable( 'bias3',
          shape = [dep3],
          initializer=tf.constant_initializer(0.0))
        stride_row=2;stride_col=2
        if batch_norm:
            conv3 = conv2d_norm(conv2,wc3,bc3, phase_train,stride_row,stride_col )
            conv3 = maxpool2d(conv3, 2,1)  
        else:
            conv3 = conv2d(conv2, wc3, bc3,stride_row,stride_col) 
            conv3 = maxpool2d(conv3,2,1)  
        variable_summaries(wc3)
        variable_summaries(bc3)     
    with tf.variable_scope('eigen_out'):
        wfc = tf.get_variable('wfc',shape=[13*2 *dep3   , dep3],
               initializer = tf.contrib.layers.xavier_initializer_conv2d(uniform=True, seed=None, dtype=tf.float32))  
        bfc=tf.get_variable( 'bfc',
          shape = [ dep3],
          initializer=tf.constant_initializer(0.0)),  
        # fully connected layer
        fc0 = tf.reshape(conv2, [-1,  int(prod(conv2.get_shape()[1:])) ]) 
        fc1= tf.nn.relu(tf.add(tf.matmul(fc0, wfc), bfc))   
        variable_summaries(wfc)
        variable_summaries(bfc) 
    
    with tf.variable_scope('Final_out'):
        wout= tf.get_variable('wout',shape=[ dep3, n_classes],
               initializer = tf.contrib.layers.xavier_initializer_conv2d(uniform=True, seed=None, dtype=tf.float32))  
        bout=tf.get_variable( 'bout',shape = [n_classes],initializer=tf.constant_initializer(0.0)) 
        # fully connected layer 
        fc2=tf.cond( phase_train, lambda: tf.nn.dropout(fc1,keep_prob=keep_prob if dropout else 1.0),lambda:fc1 ) 
        fc2=tf.nn.relu(tf.add(tf.matmul(fc2, wout), bout))  
    return fc2

def establish_model(): 
    global keep_prob,   learning_rate, training_iters,display_step,batch_size,layers,patience 
    global train_data,train_labels,eval_data,eval_labels,testX_true,testY_true , testX_pred,testY_pred , testX_no,testY_no 
    tf.reset_default_graph()
    x = tf.placeholder(tf.float32, [None, 214,30,1])
    y = tf.placeholder(tf.int32, [None, n_classes])  
    phase_train = tf.placeholder(tf.bool, name='phase_train')
        # Construct model
    global_step = tf.Variable(0, trainable=False)   
    pred = conv_net(  x, norm,phase_train) 
    prob = tf.nn.softmax(pred)
    predict_op=tf.argmax(tf.nn.softmax(pred), 1) 
    step=1
    loss_list=[]
    train_rate=[]
    eval_rate=[]
    n_incr_num =0
    best_loss = np.Inf
    # Define loss and optimizer 
    with tf.name_scope('loss'): 
        l2 = lambda_loss_amount * sum(
        tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables() )  
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=y))+l2  
    with tf.name_scope('Optimizer'):  
        optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate , decay=0.9).minimize(cost, global_step=global_step) 
    with tf.name_scope('err'): 
        correct = tf.equal(predict_op, tf.argmax(y, 1))
        err=1- tf.reduce_mean(tf.cast(correct, tf.float32))  
    tf.summary.scalar('err',err)
    tf.summary.scalar('loss',cost)
    
    #save
    saver = tf.train.Saver() 
    sess = tf.InteractiveSession() 
     
    # Initializing the variables 
    init = tf.global_variables_initializer();sess.run(init)
    if train_data.shape[1]!= 214:
        train_data = np.transpose(train_data, [0,2, 1]) 
        train_data = np.expand_dims(train_data, 3)
    if eval_data.shape[1]!= 214:
        eval_data = np.transpose(eval_data,[0,2,1])
        eval_data = np.expand_dims(eval_data,3) 
    if testX_pred.shape[1]!= 214:
        testX_pred = np.transpose(testX_pred,[0,2,1])
        testX_pred = np.expand_dims(testX_pred,3)
    if testX_no.shape[1]!= 214:
        testX_no = np.transpose(testX_no,[0,2,1])
        testX_no = np.expand_dims(testX_no,3)
    while step < training_iters: 
        ind = np.arange(train_data.shape[0])
        batch_idx = np.random.choice(ind, batch_size, replace=False)
        batch_x = train_data[batch_idx] 
        batch_y= train_labels[batch_idx] 
        indeval = np.arange(eval_data.shape[0]) 
        eval_idx = np.random.choice(indeval, batch_size, replace=False)
        batch_xeval=eval_data[eval_idx] 
        batch_yeval=eval_labels[eval_idx]  
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,phase_train:True }) 
        loss, train_err = sess.run([cost, err], feed_dict={x: batch_x, y: batch_y ,phase_train:True})
        loss_eval,eval_err=sess.run([cost ,err ], feed_dict={x: batch_xeval,  y: batch_yeval ,phase_train:True})
        loss_list.append(loss)
        train_rate.append(train_err)
        eval_rate.append(eval_err)
        step += 1
        if step % display_step == 0:
            # Calculate batch loss and err 
            print("Iter " + str(step ) + ", Minibatch Loss= " +  "{:.2f}".format(loss) + ",training err= " + "{:.2f}".format(train_err)+ ",validating err= " + "{:.2f}".format(eval_err))   
            if loss_eval < best_loss:
                best_loss = loss_eval
                n_incr_num =0
            else:
                n_incr_num+=1
            if (n_incr_num >= patience) and (step > 3000):
                print ('Early_stopping! and the iterations is', step)
                # Create the collection.
                tf.get_collection("validation_nodes")
                # Add stuff to the collection.
                tf.add_to_collection("validation_nodes", x)  
                tf.add_to_collection("validation_nodes", y) 
                tf.add_to_collection("validation_nodes", phase_train) 
                tf.add_to_collection("validation_nodes", prob)  
                save_path = saver.save(sess, savepath+model_name) 
                correct_results_pred=sess.run(correct , feed_dict={x:testX_pred,  y:testY_pred,phase_train:True}) 
                total_test_pred=sess.run(err , feed_dict={x:testX_pred,  y:testY_pred,phase_train:True}) 
                correct_results_no=sess.run(correct , feed_dict={x:testX_no,   y:testY_no,phase_train:True}) 
                total_test_no=sess.run(err , feed_dict={x:testX_no,  y:testY_no,phase_train:True})    
                print("Testing err of subtract predicted :", total_test_pred) 
                print("Testing err of subtract nothing :", total_test_no)  
                return loss_list,step,train_rate,eval_rate, correct_results_pred,correct_results_no 
    # Create the collection.
    tf.get_collection("validation_nodes")
    # Add stuff to the collection.
    tf.add_to_collection("validation_nodes", x) 
    tf.add_to_collection("validation_nodes", y) 
    tf.add_to_collection("validation_nodes", phase_train) 
    tf.add_to_collection("validation_nodes", prob) 
    save_path = saver.save(sess, savepath+model_name) 
    correct_results_pred=sess.run(correct , feed_dict={x:testX_pred,   y:testY_pred,phase_train:True}) 
    total_test_pred=sess.run(err , feed_dict={x:testX_pred,   y:testY_pred,phase_train:True}) 
    correct_results_no=sess.run(correct , feed_dict={x:testX_no,  y:testY_no,phase_train:True}) 
    total_test_no=sess.run(err , feed_dict={x:testX_no,   y:testY_no,phase_train:True}) 
    print("Testing err of subtract predicted :", total_test_pred) 
    print("Testing err of subtract nothing :", total_test_no)  
    print("Optimization Finished!") 
    return loss_list,step,train_rate,eval_rate,  correct_results_pred,correct_results_no  

def each_perform(correct_results,eval_labels ):
    label_y=np.argmax(eval_labels,1)
    
    zero = np.where(label_y==0)
    correct=[correct_results[i] for i in zero]
    accuracy_zero=1-np.mean(correct)

    one = np.where(label_y==1)
    correct=[correct_results[i] for i in one]
    accuracy_one=1-np.mean(correct)

    two = np.where(label_y==2)
    correct=[correct_results[i] for i in two]
    accuracy_two=1-np.mean(correct) 
    
   
    print(100-100*accuracy_zero)
    print(100-100*accuracy_one)
    print(100-100*accuracy_two)  

def plot_loss(loss,train_step,from_second,name_save, plot_name,plot_title):
    if from_second :
        plt.plot(range(0,train_step-1,1),loss[1:])
    else:
        plt.plot(range(0,train_step,1),loss[0:])
    plt.xlabel('Iterative times (t)')
    plt.ylabel(plot_name)
    plt.title(plot_title)
    plt.grid(True) 
    plt.show() 
    

In [ ]:
train=1
if train==1:  
    start_time = time.time()
    loss ,step,train_rate,eval_rate,  correct_results_pred,correct_results_no=establish_model()    
    eclapse = time.time() -start_time
    print ('Running time is',eclapse)
    plot_loss(loss,step-1,False,"Loss_value.png", 'Loss','Loss function value with iterations') 
    plot_loss(train_rate,step-1,False,"Train_err_rate.png",'Training accurate rate','Training classification with iterations')
    plot_loss(eval_rate,step-1,False,"Eval_err_rate.png",'Evaluating accurate rate','Evaluating classification with iterations')  
    